In [1]:
import torch
from torchvision import datasets

In [2]:
# download the FMNIST training dataset
fmnist = datasets.FashionMNIST('~/data/FMNIST', download=True, train = True)

# fmnist validation dataset
val_fmnist = datasets.FashionMNIST('~/data/FMNIST', train = False)

100%|██████████| 26421880/26421880 [00:01<00:00, 16196301.76it/s]


Extracting /root/data/FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/data/FMNIST/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 268403.96it/s]


Extracting /root/data/FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/data/FMNIST/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5061139.83it/s]


Extracting /root/data/FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/data/FMNIST/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4369137.39it/s]

Extracting /root/data/FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/data/FMNIST/FashionMNIST/raw



In [3]:
# training dataset
tr_images  = fmnist.data

# validation datasets
val_images = val_fmnist.data

# training labels
tr_targets = fmnist.targets

# validation labels
val_targets = val_fmnist.targets

In [6]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
# build a class that fetch the dataset
class MyFMNISTDataset(Dataset):
    def __init__(self, x, y):
        
        # convert to float values, flatten
        # store them into the device
        x = x.float()/255.
        x = x.view(-1, 1, 28,28)         #(n_samples, c_channels, height, width)
        self.x = x.to(device)
        self.y = y.to(device)
        
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    
    def __len__(self):
        return len(self.x)

In [8]:
# build a function to fetch a batchsize from the dataset
def get_data():
    # fetch the training dataset and the batches from itself
    trn_dts = MyFMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(trn_dts, batch_size=32, shuffle=True)
    
    # fetch the validation dataset and the batches from itself
    val_dts = MyFMNISTDataset(val_images, val_targets)
    val_dl = DataLoader(val_dts, batch_size=len(val_images), shuffle=False)
    return trn_dl, val_dl

In [9]:
from torch.optim import SGD, Adam

# define a model function
def get_model():

    my_model = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(3200, 256), 
        nn.ReLU(),
        nn.Linear(256, 10)
    ).to(device)
    
    #  Loss function
    loss_fn = nn.CrossEntropyLoss()
    
    # optimizer
    optimizer = Adam(my_model.parameters(), lr = 0.001)
    
    return my_model, loss_fn, optimizer

In [10]:
# Train the dataset on images batches
def train_batch(batch_x, batch_y, my_model, loss_fn, optimizer):
    # train mode
    my_model.train()
    
    # flush the previous gradients
    optimizer.zero_grad()
    
    # compute the batch loss
    batch_loss = loss_fn(my_model(batch_x), batch_y)
    
    # backward pass
    batch_loss.backward()
    
    # update parameters
    optimizer.step()
    
    return batch_loss.item()

In [11]:
@torch.no_grad()
def val_loss(val_batch_x, val_batch_y, my_model):
    # evaluation mode
    my_model.eval()
    
    # compute validation loss
    val_batch_loss = loss_fn(my_model(val_batch_x), val_batch_y)
    return val_batch_loss.item()

In [12]:
# compute the accuracy

# disable gradient computation in the whole function
@torch.no_grad()
def accuracy(batch_x, batch_y, my_model):
    
    # evaluation mode
    my_model.eval()
    
    # compute the max and the argmax
    max_values, argmaxes = my_model(batch_x).max(-1)
    
    # coincide with ground truth
    is_correct = argmaxes == batch_y
    
    return is_correct.cpu().numpy().tolist()

In [13]:
!pip install torch_summary
from torchsummary import summary
my_model, loss_fn, optimizer = get_model()
summary(my_model, torch.zeros(1, 1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 26, 26]          640
├─MaxPool2d: 1-2                         [-1, 64, 13, 13]          --
├─ReLU: 1-3                              [-1, 64, 13, 13]          --
├─Conv2d: 1-4                            [-1, 128, 11, 11]         73,856
├─MaxPool2d: 1-5                         [-1, 128, 5, 5]           --
├─ReLU: 1-6                              [-1, 128, 5, 5]           --
├─Flatten: 1-7                           [-1, 3200]                --
├─Linear: 1-8                            [-1, 256]                 819,456
├─ReLU: 1-9                              [-1, 256]                 --
├─Linear: 1-10                           [-1, 10]                  2,570
Total params: 896,522
Trainable params: 896,522
Non-trainable params: 0
Total mult-adds (M): 10.13
Input size (MB

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 26, 26]          640
├─MaxPool2d: 1-2                         [-1, 64, 13, 13]          --
├─ReLU: 1-3                              [-1, 64, 13, 13]          --
├─Conv2d: 1-4                            [-1, 128, 11, 11]         73,856
├─MaxPool2d: 1-5                         [-1, 128, 5, 5]           --
├─ReLU: 1-6                              [-1, 128, 5, 5]           --
├─Flatten: 1-7                           [-1, 3200]                --
├─Linear: 1-8                            [-1, 256]                 819,456
├─ReLU: 1-9                              [-1, 256]                 --
├─Linear: 1-10                           [-1, 10]                  2,570
Total params: 896,522
Trainable params: 896,522
Non-trainable params: 0
Total mult-adds (M): 10.13
Input size (MB): 0.00
Forward/backward pass size (MB): 0.45
Params size (MB): 3.42
Estimated Total Size (MB): 3.8

In [14]:
trn_dl, val_dl = get_data()

In [15]:
import numpy as np
from torch.optim import lr_scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.8, patience = 0, 
                                           threshold = 0.001, min_lr = 0.00001,
                                           threshold_mode='abs',
                                           verbose=True)
# empty list of the total losses
losses, accuracies = [], []
val_losses, val_accuracies = [], []

for epoch in range(5):
    batch_losses, batch_accuracies = [], []
    
    for ix, (batch_x, batch_y) in enumerate(iter(trn_dl)):
        
        batch_loss = train_batch(batch_x, batch_y, my_model, loss_fn, optimizer)
        
        batch_losses.append(batch_loss)
    train_epoch_loss = np.array(batch_losses).mean()
    
    for ix, batches in enumerate(iter(trn_dl)):
        batch_x, batch_y = batches
        
        batch_accuracy = accuracy(batch_x, batch_y, my_model)
        
        batch_accuracies.extend(batch_accuracy)
    train_epoch_accuracy = np.array(batch_accuracies).mean()
    
    # Validation set
    for ix, (val_batch_x, val_batch_y) in enumerate(iter(val_dl)):
        val_batch_accuracies = accuracy(val_batch_x, val_batch_y, my_model)
        val_batch_loss = val_loss(val_batch_x, val_batch_y, my_model)
        scheduler.step(val_batch_loss)
    val_epoch_accuracy = np.array(val_batch_accuracies).mean()
        
    losses.append(train_epoch_loss)
    accuracies.append(train_epoch_accuracy)
    val_losses.append(val_batch_loss)
    val_accuracies.append(val_epoch_accuracy)
    print('Epoch =', str(epoch+1) + ' is done')

Epoch = 1 is done
Epoch = 2 is done
Epoch = 3 is done
Epoch 00004: reducing learning rate of group 0 to 8.0000e-04.
Epoch = 4 is done
Epoch 00005: reducing learning rate of group 0 to 6.4000e-04.
Epoch = 5 is done
